In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

bart = pd.read_csv("/content/drive/MyDrive/newssumm/df_eval_after_bart.csv")
peg  = pd.read_csv("/content/drive/MyDrive/newssumm/df_eval_after_bart_pegasus.csv")
led  = pd.read_csv("/content/drive/MyDrive/newssumm/led_inference.csv")

# take only first 25 rows (as decided)
bart = bart.iloc[:25].reset_index(drop=True)
peg  = peg.iloc[:25].reset_index(drop=True)
led  = led.iloc[:25].reset_index(drop=True)

print(bart.shape, peg.shape, led.shape)


(25, 6) (25, 6) (25, 6)


In [5]:
print(bart.columns)
print(peg.columns)
print(led.columns)


Index(['input_text', 'target_summary', 'num_docs', 'category', 'date',
       'bart_summary'],
      dtype='object')
Index(['input_text', 'target_summary', 'num_docs', 'category', 'date',
       'pegasus_summary'],
      dtype='object')
Index(['input_text', 'target_summary', 'num_docs', 'category', 'date',
       'led_summary'],
      dtype='object')


In [6]:
compare_df = pd.DataFrame({
    "input_text": bart["input_text"],
    "reference_summary": bart["target_summary"],
    "bart_summary": bart["bart_summary"],
    "pegasus_summary": peg["pegasus_summary"],
    "led_summary": led["led_summary"],
})

compare_df.head()


,input_text,reference_summary,bart_summary,pegasus_summary,led_summary
0,[DOC_1]\nDefence Minister Rajnath Singh on Wed...,Defence Minister Rajnath Singh said that the ...,Defence Minister Rajnath Singh on Wednesday sa...,The clash in Galwan Valley has raised concerns...,Defence Minister Rajnath Singh on Wednesday sa...
1,[DOC_1]\nThe Guwahati Municipal Corporation (G...,The famous Ravan Dahan event at Central Sports...,The Guwahati Municipal Corporation (GMC) has l...,The Guwahati Municipal Corporation (GMC) has l...,The Guwahati Municipal Corporation (GMC) launc...
2,"[DOC_1]\nA day after US President Joe Biden, B...",With the help of open-source information and s...,"India's Prime Minister Rishi Sunak, who swept ...",British Prime Minister Rishi Sunak met his Isr...,"A day after US President Joe Biden, British Pr..."
3,[DOC_1]\nThe Akhil Bhartiya Vanvasi Kalyan Ash...,The Union Ministry of Power in India has infor...,The Akhil Bhartiya Vanvasi Kalyan Ashram has u...,The Akhil Bhartiya Vanvasi Kalyan Ashram has u...,The Akhil Bhartiya Vanvasi Kalyan Ashram has u...
4,[DOC_1]\nMumbai: As India approaches the May 3...,Some experts believe it's time to end the lock...,Experts from the National Task Force of the In...,As India approaches the May 3 deadline for lif...,The lockdown effort was a welcome move by the ...


In [7]:

compare_df.to_csv(
    "/content/drive/MyDrive/newssumm/final_comparison_25.csv",
    index=False
)

print("Saved final_comparison_25.csv")


Saved final_comparison_25.csv


In [9]:
!pip install -q evaluate rouge-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00


In [10]:
import evaluate
rouge = evaluate.load("rouge")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
bart_scores = rouge.compute(
    predictions=compare_df["bart_summary"].tolist(),
    references=compare_df["reference_summary"].tolist()
)

pegasus_scores = rouge.compute(
    predictions=compare_df["pegasus_summary"].tolist(),
    references=compare_df["reference_summary"].tolist()
)

led_scores = rouge.compute(
    predictions=compare_df["led_summary"].tolist(),
    references=compare_df["reference_summary"].tolist()
)

bart_scores, pegasus_scores, led_scores


({'rouge1': np.float64(0.301961781965906),
  'rouge2': np.float64(0.1231805091044661),
  'rougeL': np.float64(0.20685097596367447),
  'rougeLsum': np.float64(0.2144714951303277)},
 {'rouge1': np.float64(0.23676217905519914),
  'rouge2': np.float64(0.07857503940884825),
  'rougeL': np.float64(0.1645895118817638),
  'rougeLsum': np.float64(0.1678979374693136)},
 {'rouge1': np.float64(0.23437650425606385),
  'rouge2': np.float64(0.07406832416286083),
  'rougeL': np.float64(0.1691489808572728),
  'rougeLsum': np.float64(0.17761805390755997)})

In [12]:
import pandas as pd

rouge_table = pd.DataFrame({
    "Model": ["BART", "PEGASUS", "LED"],
    "ROUGE-1": [bart_scores["rouge1"], pegasus_scores["rouge1"], led_scores["rouge1"]],
    "ROUGE-2": [bart_scores["rouge2"], pegasus_scores["rouge2"], led_scores["rouge2"]],
    "ROUGE-L": [bart_scores["rougeL"], pegasus_scores["rougeL"], led_scores["rougeL"]],
    "ROUGE-Lsum": [bart_scores["rougeLsum"], pegasus_scores["rougeLsum"], led_scores["rougeLsum"]],
})

rouge_table


,Model,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-Lsum
0,BART,0.301962,0.123181,0.206851,0.214471
1,PEGASUS,0.236762,0.078575,0.164590,0.167898
2,LED,0.234377,0.074068,0.169149,0.177618


In [13]:
rouge_table.to_csv("/content/drive/MyDrive/newssumm/rouge_comparison_25.csv", index=False)
